In [17]:
import pickle

path = "Textual features\\Tweets\\tweets"

with open(path, 'rb') as f:
  X = pickle.load(f)

In [42]:
import numpy as np
import functools
import operator

data = []

for vec in X.values():
    data.append(vec)

X_ = functools.reduce(operator.iconcat, data, [])

In [43]:
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 10


input_vec = keras.Input(shape=(384,))
# "encoded" is the encoded representation of the input
layer_1 = layers.Dense(182)(input_vec)
layer_2 = layers.Dense(90)(layer_1)
layer_3 = layers.Dense(40)(layer_2)
encoded = layers.Dense(encoding_dim, activation='relu')(layer_3)

# "decoded" is the lossy reconstruction of the input
layer_4 = layers.Dense(40)(encoded)
layer_5 = layers.Dense(90)(layer_4)
layer_6 = layers.Dense(182)(layer_5)
decoded = layers.Dense(384, activation='sigmoid')(layer_6)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_vec, decoded)
encoder = keras.Model(input_vec, encoded)

# encoded_input = keras.Input(shape=(encoding_dim,))

autoencoder.compile(optimizer="adam", loss="mse")

In [44]:
from sklearn.model_selection import train_test_split

x_train, x_train_t, x_test, x_test_t = train_test_split(X_, X_, test_size=0.1, random_state=42)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

autoencoder.fit(np.array(x_train), np.array(x_test),
                epochs=200,
                batch_size=64,
                shuffle=True,
                validation_data=(np.array(x_train_t), np.array(x_test_t)),
                callbacks=[callback],
                verbose=1)



In [46]:
autoencoder.save("Textual features\\Tweets\\autoencoder.h5")